In [1]:
# Data Processing & Feature Engineering
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

# Model Selection & Evaluation
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

# Tree-based Models
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Deep Learning (RNN, LSTM) - TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Dropout

# Visualization
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns

# Custom Modules
from data_preprocessing import load_data, apply_feature_engineering

In [2]:
df = load_data()

In [3]:
df_eng = apply_feature_engineering(df)

Starting feature engineering...
Converting date columns...
Extracting travel duration...
Imputing missing travel distances...
Processing departure times...
Processing airline and cabin class codes...
Applying Label Encoding...
Label Encoding complete!
Calculating days to departure...
Processing holiday features...
Dropping unnecessary columns...
Feature engineering complete!


In [4]:
df_eng.head()

,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isRefundable,isNonStop,totalFare,seatsRemaining,airlineCode,cabinClass,travelDistance,departureTimeHour,daysToDeparture,departureDayOfWeek,isWeekend,isHoliday,nearHoliday
0,2022-04-16,2022-04-17,0,1,149,False,True,248.6,9,70,42,947.0,12,1,6,True,True,True
1,2022-04-16,2022-04-17,0,1,150,False,True,248.6,4,70,42,947.0,06,1,6,True,True,True
2,2022-04-16,2022-04-17,0,1,150,False,True,248.6,9,70,42,947.0,11,1,6,True,True,True
3,2022-04-16,2022-04-17,0,1,152,False,True,248.6,8,70,42,947.0,13,1,6,True,True,True
4,2022-04-16,2022-04-17,0,1,154,False,True,248.6,9,70,42,947.0,09,1,6,True,True,True
